# PRESENTATION

this notebooks will test the rating meta model developped by Remy Coulom.
It is a strong alternative to ELO or Glicko traditional models used to rate players in Chess.
https://hal.inria.fr/inria-00323349/document/

WHR0 is the python implementation from here: 
https://github.com/pfmonville/whole_history_rating

WHR1 is from here:
https://github.com/wind23/whr_renju
https://renjunews.com/whole-history-rating-whr-and-renju-rating/

this notebook will apply the rating model to ATP tennis matches.

In [1]:
#pip install whole-history-rating

In [2]:
import pandas as pd
import statistics
import numpy as np

In [3]:
def get_query_surface(index_surface: str)->str:
    """Return the clause of the query for the surface
    Example: 'CourtId=="Clay" '
    Parameters
    ----------
    index_surface : str
        And index between 0 and 4 for the surface
        (NonClay=0,Clay=1,Hard=2,Indoor=3,Grass=4)
    Raises
    ------
    Exception
        If no sound is set for the animal or passed in as a
        parameter.
    """
    index_surface = str(index_surface)
    str_clause_surface = ''
    if (index_surface =='1'):
        strClauseSurface = 'CourtId=="2" '
    elif (index_surface =='0'):
        strClauseSurface = 'CourtId!="2" '
    elif (index_surface =='2'):
        strClauseSurface = 'CourtId=="1" '
    elif (index_surface =='3'):
        strClauseSurface = 'CourtId=="3" '
    elif (index_surface =='4'):
        strClauseSurface = 'CourtId=="5" '
    else :
        raise Exception("indexSurface is incorrect for get_query_surface(indexSurface)")# + get_query_surface.__doc__)
    return strClauseSurface

# LOAD Data

into 2 dataframes:
- **df**: will contain all matches
- **df9mCourt**: will contain all matches for the selected court(surface) and only for the last 9 months

In [4]:
#LOAD DATA
yrstart=2017
yrend=2021
dfMatches=pd.DataFrame()
for year in range(yrend, yrstart-1,-1):
    #dfcha=pd.read_csv((r"https://raw.githubusercontent.com/damienld/tennis_atp/master/atp_matches_qual_chall_{0}.csv").format(str(year)),index_col=0)
    dfyear=pd.read_csv(("./Data/ATP{0}_all_matches.csv").format(str(year)), parse_dates = ['Date'])
    dfMatches=pd.concat([dfMatches,dfyear],axis=0)

dfMatches = dfMatches.sort_values(by="Date", ascending=False)    
colP1id="P1"
colP2id="P2"
colDate="Date"
colCourt="CourtId"
#colResult="IndexP"

df=dfMatches[[colP1id,colP2id,colDate,colCourt]]
df=df.drop_duplicates()
df=df
df["RES"]="B"# if val==0 else "B" for val in X[colResult]]
Date0=df.iloc[-1,:][colDate]
print("Since " + str(Date0))
df["Days"]=(df[colDate]-Date0).dt.days+1#-1791+270
print('nb matches: ' + str(len(df))+"\n")
df


Since 2016-12-29 00:00:00
nb matches: 56000



,P1,P2,Date,CourtId,RES,Days
1029,Carlos Alcaraz Garfia,Sebastian Baez,2021-12-11,3,B,1809
1053,Jack Draper,Federico Gaio,2021-12-11,3,B,1809
1055,Jaume Antoni Munar Clar,Camilo Ugo Carabelli,2021-12-11,2,B,1809
1056,Juan Ignacio Londero,Thiago Moura Monteiro,2021-12-11,2,B,1809
1058,Manuel Guinard,Andrea Arnaboldi,2021-12-11,3,B,1809
...,...,...,...,...,...,...
25284,David Goffin,Andy Murray,2016-12-30,1,B,2
25286,Rafael Nadal,Milos Raonic,2016-12-30,1,B,2
25288,Jo-Wilfried Tsonga,Tomas Berdych,2016-12-30,1,B,2
25290,David Goffin,Jo-Wilfried Tsonga,2016-12-29,1,B,1


In [5]:
import datetime

#USER can select the surface which will be used for the last 9 monts court dataframe
dict_surfaces = {0: "NonClay", 1:"Clay", 2:"Hard", 3: "Indoor",4: "Grass"}
print("SELECT surface:" + str(dict_surfaces))
index_sel = int(input())
print(dict_surfaces.get(index_sel))
strClauseSurface = get_query_surface(index_sel)

print("Last9M")
#FILTER the dataframe for only last 9 months
df9mCourt=dfMatches[[colP1id,colP2id,colDate,colCourt]]
df9mCourt=df9mCourt.drop_duplicates()
df9mCourt["RES"]="B"# if val==0 else "B" for val in X[colResult]]
df9mCourt=df9mCourt[df9mCourt[colDate] >= datetime.datetime.now() - pd.to_timedelta("275day")]
Date0=df9mCourt.iloc[-1,:][colDate]
print("Since " + str(Date0))
df9mCourt["Days"]=(df9mCourt[colDate]-Date0).dt.days+1#-1791+270

#FILTER the dataframe for only the selected surface/court
df9mCourt=df9mCourt.query(strClauseSurface)
print('nb matches: ' + str(len(df9mCourt)))
df9mCourt.drop("CourtId", axis=1)
df9mCourt

SELECT surface:{0: 'NonClay', 1: 'Clay', 2: 'Hard', 3: 'Indoor', 4: 'Grass'}
0
NonClay
Last9M
Since 2021-03-03 00:00:00
nb matches: 8851


,P1,P2,Date,CourtId,RES,Days
1029,Carlos Alcaraz Garfia,Sebastian Baez,2021-12-11,3,B,284
1053,Jack Draper,Federico Gaio,2021-12-11,3,B,284
1055,Jaume Antoni Munar Clar,Camilo Ugo Carabelli,2021-12-11,2,B,284
1056,Juan Ignacio Londero,Thiago Moura Monteiro,2021-12-11,2,B,284
1058,Manuel Guinard,Andrea Arnaboldi,2021-12-11,3,B,284
...,...,...,...,...,...,...
17658,Albert Ramos-Vinolas,Dominik Koepfer,2021-03-03,2,B,1
17660,Alessandro Giannessi,Nikolas Sanchez-Izquierdo,2021-03-03,2,B,1
17662,Alex Molcan,Enzo Couacaud,2021-03-03,2,B,1
17664,Altug Celikbilek,Alexey Vatutin,2021-03-03,3,B,1


# WHR model

import and instantiate the models **whr** and **whr9m**

For each of them, create all matches from the dataframes into the whr instance

In [6]:
import sys
import os
import pytest

#sys.path.append(os.path.join(os.path.dirname(__file__), "./"))
from whr import whole_history_rating
from whr import utils
"""
One of the meta parameters to WHR is the variance of rating change over one time step, :w2, which determines how much that a player's rating is likely change in one day. Higher numbers allow for faster progress. The default value is 300, which is fairly high.
Rémi Coulom in his paper, used w2=14 to get his results"""

#create an instance
whr = whole_history_rating.Base({'w2':10})
#add each game from the dataframe
[whr.create_game(p1, p2, res, days, 0) for (p1, p2, res, days) in zip(df[colP1id], df[colP2id], df["RES"], df["Days"])]
# Base.create_game() arguments: black player name, white player name, winner, day number, handicap
# Handicap should generally be less than 500 elo

#create an instance
whr9m = whole_history_rating.Base({'w2':1})
#add each game from the dataframe
[whr9m.create_game(p1, p2, res, days, 0) for (p1, p2, res, days) in zip(df9mCourt[colP1id], df9mCourt[colP2id], df9mCourt["RES"], df9mCourt["Days"])]


 ...]

**ITERATE** the WHR models

In [7]:
# Iterate the WHR algorithm towards convergence with more players/games, more iterations are needed.
#whr.iterate(50)

# Or let the module iterate until the elo is stable (precision by default 10E-3) with a time limit of 10 seconds by default
whr.auto_iterate(time_limit = 10, precision = 10E-3)
whr9m.auto_iterate(time_limit = 10, precision = 10E-3)

(40, False)

In [8]:
# You can print or get all ratings ordered
#whr.print_ordered_ratings(current=False) # current to True to only get the last rank estimation
#whr.get_ordered_ratings(current=False, compact=False) # compact to True to not have the name before each ranks
whr.print_ordered_ratings(current=True)

3304 Andrey Yakovlev => -540.8546113836477
3303 Alexey Kedryuk => -514.4404879685067
3302 Goran Markovic => -490.781896648535
3301 Petar Pjevcevic => -489.1962893155537
3300 Daniil Savelev => -488.4182167699277
3299 Isaac Stoute => -470.6638515242773
3298 Sagadat Ayap => -467.72779471222833
3297 Tomasz Bednarek => -461.490227797548
3296 Casey Hoole => -459.7163921816513
3295 Anton Chekhov => -449.08544940612404
3294 Alexander Pavlioutchenkov => -448.67765502864586
3293 Tobias Sonne => -447.7699154281596
3292 Ivar Aramburu Contreras => -429.7337331933361
3291 Yair Sarouk => -427.112474920152
3290 Batyr Sapaev => -418.10093521529654
3289 Winter Meagher => -409.8942199857841
3288 Milan Radojkovic => -401.76864343085134
3287 Courage Okungbowa => -401.38903365632086
3286 Sean Hodkin => -401.1040057355099
3285 Vladan Tadic => -388.0534900744475
3284 Aaron Addison => -380.7412602680521
3283 James Cerretani => -379.64620500905164
3282 Roberto Maytin => -378.6232884281214
3281 Tai Sach => -376.

840 Alexander Brown => 32.606601068259664
839 Joao Monteiro => 32.742566057584995
838 Lorenzo Battista => 32.75102331023558
837 Kaito Uesugi => 33.2687603414762
836 Matej Sabanov => 33.27740332948723
835 Duckhee Lee => 33.39243217244366
834 Rodrigo Banzer => 33.64468633481172
833 Pol Wattanakul => 33.65386729208366
832 Eric Hadigian => 33.6743600261309
831 Antonio Sancic => 33.75038435972615
830 Filip Peliwo => 34.17932094920916
829 Andrew Dale => 34.21628576209824
828 Corey Gaal => 34.337097511781785
827 Ben Patael => 34.36127220622938
826 Gabriele Piraino => 34.40316060746899
825 Janos Fekete => 34.43963916683076
824 Jakob Aichhorn => 34.76729117260276
823 Jonas Maier => 34.894144845878536
822 Andres Artunedo Martinavarr => 34.89630329931395
821 Hamad Medjedovic => 34.899034190069735
820 Kenny de Schepper => 34.974566987141955
819 Yuki Mochizuki => 35.24353636806953
818 Tomas Liska => 36.05954010747684
817 Tao Mu => 36.065199231413544
816 Viacheslav Bielinskyi => 36.09175461053973
81

In [9]:
# Results are stored in one triplet for each game: [day_number, elo_rating, uncertainty]
whr9m.ratings_for_player("Borna Gojo")

[[37, 79, 12],
 [39, 79, 12],
 [49, 79, 12],
 [76, 80, 12],
 [83, 80, 12],
 [85, 80, 12],
 [96, 80, 12],
 [103, 80, 12],
 [104, 80, 12],
 [112, 80, 12],
 [126, 80, 12],
 [135, 80, 12],
 [136, 80, 12],
 [137, 80, 12],
 [140, 80, 12],
 [157, 81, 12],
 [175, 81, 12],
 [190, 81, 12],
 [195, 81, 12],
 [197, 81, 12],
 [199, 81, 12],
 [200, 81, 12],
 [204, 81, 12],
 [213, 82, 12],
 [223, 82, 12],
 [228, 82, 12],
 [229, 82, 12],
 [230, 82, 12],
 [248, 82, 12],
 [254, 82, 12],
 [268, 82, 12],
 [272, 82, 12],
 [274, 82, 12],
 [278, 82, 12],
 [280, 82, 12],
 [283, 82, 12]]

In [10]:
#SELECT 2 Players from inputted substrings
def get_player_from_inputted_substring():
    print("Substring player name:")
    #get the list of all players
    lst_players=list(whr.players)
    #user inputs a string to search
    str_search = input()
    found_players=list(s for s in lst_players if str_search in s)
    if len(found_players) == 0:
        print("Not Found")
        return
    elif len(found_players) == 1:
        print("Found:"+ found_players[0])
        return found_players[0]
    else:
        #create a dict with all player names and an index
        indexes = list(range(len(found_players)))
        dict_indexes_values = {indexes[i]: found_players[i] for i in range(0, len(found_players), 1)} 
        print(dict_indexes_values)
        index_sel = int(input())
        return dict_indexes_values.get(index_sel)
    
print("P1")    
p1=get_player_from_inputted_substring()
print("=> "+p1)
print("\nP2")    
p2=get_player_from_inputted_substring()
print("=> "+p2)
print("\nPREDICTION:")
# You can get a prediction for a future game between two players (even non existing players)
# Base.probability_future_match() arguments: black player name, white player name, handicap
whr.probability_future_match(p1,p2, 0)
print("Last9 months:")
whr9m.probability_future_match(p1,p2, 0)

P1
Substring player name:
a Gojo
Found:Borna Gojo
=> Borna Gojo

P2
Substring player name:
Krajino
Found:Filip Krajinovic
=> Filip Krajinovic

PREDICTION:
win probability: Borna Gojo:27.68%; Filip Krajinovic:72.32%
Last9 months:
win probability: Borna Gojo:26.29%; Filip Krajinovic:73.71%


(0.26285154924775156, 0.7371484507522486)

 You can load several games all together using a file or a list of string representing the game
 all elements in list must be like: "black_name white_name winner time_step handicap extras" 
 you can exclude handicap (default=0) and extras (default={})
whr.load_games(["shusaku shusai B 1 0", "shusaku shusai W 2", "shusaku shusai W 3 0"])
whr.load_games(["firstname1 name1, firstname2 name2, W, 1"], separator=",")

 You can save and load a base (you don't have to redo all iterations)
whr.save_base(path)
whr2 = whole_history_rating.Base.load_base(path)

import sys
!conda install --yes --prefix {sys.prefix} whole-history-rating

import sys
!conda install --yes --prefix {sys.prefix} tkinter